In [ ]:
# ---- CELL 1: Install dependencies ----
%pip install /lakehouse/default/Files/fabric_api-0.4.0-py3-none-any.whl
%pip install sparkdantic

In [2]:
# ---- CELL 2: Environment setup ----
import os
import logging
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Set credentials from Key Vault (in production) or environment
os.environ["CW_AUTH_USERNAME"] = "thekking+yemGyHDPdJ1hpuqx"
os.environ["CW_AUTH_PASSWORD"] = "yMqpe26Jcu55FbQk"
os.environ["CW_CLIENTID"] = "c7ea92d2-eaf5-4bfb-a09c-58d7f9dd7b81"

StatementMeta(, db980de1-0d61-450c-8907-e556d3d33e05, 11, Finished, Available, Finished)

In [3]:
# ---- CELL 3: Initialize Spark and client ----
from pyspark.sql import SparkSession
from fabric_api.client import ConnectWiseClient

spark = SparkSession.getActiveSession() or SparkSession.builder.getOrCreate()
client = ConnectWiseClient()

# Test connectivity
response = client.get("/system/info")
print(f"API connected: {response.status_code == 200}")

StatementMeta(, db980de1-0d61-450c-8907-e556d3d33e05, 12, Finished, Available, Finished)

API connected: True


In [10]:
# ---- CELL 4: Define test parameters ----
# Small test dataset
test_entities = [
    "Agreement",
    "TimeEntry",
    "ExpenseEntry",
    "ProductItem",
    "PostedInvoice",
    "UnpostedInvoice",
]
page_size = 1000
max_pages = 200  # 1000 records max per entity


# Paths
bronze_path = "Tables/bronze"
silver_path = "Tables/silver"
gold_path = "Tables/gold"

StatementMeta(, db980de1-0d61-450c-8907-e556d3d33e05, 19, Finished, Available, Finished)

In [ ]:
# ---- CELL 5: Bronze layer - Overwrite mode ----
from fabric_api.pipeline import process_entity_to_bronze

print("=== BRONZE LAYER - OVERWRITE MODE ===")
bronze_results_overwrite = {}

for entity in test_entities:
    logger.info(f"Processing {entity} to bronze (overwrite)...")
    result = process_entity_to_bronze(
        entity_name=entity,
        client=client,
        page_size=page_size,
        max_pages=max_pages,
        bronze_path=bronze_path,
        mode="overwrite",
    )
    bronze_results_overwrite[entity] = result
    print(f"{entity}: {result['bronze_rows']} rows")

In [ ]:
# ---- CELL 8: Silver layer - Overwrite mode ----
from fabric_api.pipeline import process_bronze_to_silver

print("\n=== SILVER LAYER - OVERWRITE MODE ===")
silver_results_overwrite = {}

for entity in test_entities:
    logger.info(f"Processing {entity} to silver (overwrite)...")
    result = process_bronze_to_silver(
        entity_name=entity, bronze_path=bronze_path, silver_path=silver_path, spark=spark
    )
    silver_results_overwrite[entity] = result
    print(f"{entity}: {result.get('silver_rows', 0)} rows")

In [ ]:
# ---- CELL 11: Gold layer - Overwrite mode ----
from fabric_api.gold.invoice_processing import run_gold_invoice_processing

print("\n=== GOLD LAYER - OVERWRITE MODE ===")
gold_results = run_gold_invoice_processing(
    silver_path=silver_path, gold_path=gold_path, spark=spark
)

for table, count in gold_results.items():
    print(f"Gold {table}: {count} rows")